In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 450kB 18.2MB/s 
     |████████████████████████████████| 92kB 8.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import *
import pickle
from konlpy.tag import Okt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
pd.set_option('display.unicode.east_asian.width', True)

In [ ]:
with open('/content/datasets/10x10_token2.pickle', 'rb') as f:
  token = pickle.load(f) #피클에 저장해놓은 토큰 가져옴

In [ ]:
with open('/content/datasets/10x10_category_encoder.pickle', 'rb') as f:
  encoder = pickle.load(f) #피클형태로 저장해놓은 인코더 가져옴
category = encoder.classes_
print(category)  

['Baby,Kids' 'Beauty' 'Deco' 'FasihonClothing' 'FasionGoods' 'Food'
 'Furniture' 'Jewelry' 'Kitchen' 'Life']


In [ ]:
test_data_ne = pd.read_csv('/content/datasets/test_data_ne.csv', 
                            index_col=0)
print(test_data_ne.head())

                        title   category
0      윈썸 우드 롤 업 테이블  Furniture
1     모노그램 휘게 체어 블랙  Furniture
2     모노그램 휘게 체어 레드  Furniture
3   모노그램 휘게 체어 네이비  Furniture
4       모노그램    체어 세트  Furniture


In [ ]:
print(test_data_ne.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2132 entries, 0 to 2153
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     2129 non-null   object
 1   category  2132 non-null   object
dtypes: object(2)
memory usage: 50.0+ KB
None


In [ ]:
test_data_ne = test_data_ne.dropna(axis=0)

test_data_ne.reset_index(drop=True,inplace=True)

In [ ]:
sum_dup = test_data_ne.title.duplicated().sum()#중복 행 개수 확인
sum_dup

341

In [ ]:
#중복제거
test_data_ne = test_data_ne.drop_duplicates(subset=['title'])#title에서 중복값 존재하는 행 제거한 df생성
sum_dup = test_data_ne.title.duplicated().sum()#중복 행 개수 확인
print(sum_dup)

0


In [ ]:
test_data_ne.reset_index(drop=True,inplace=True)

In [ ]:
#X,Y분류
X = test_data_ne['title']
Y = test_data_ne['category']

In [ ]:
test_data_ne.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1788 entries, 0 to 1787
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     1788 non-null   object
 1   category  1788 non-null   object
dtypes: object(2)
memory usage: 28.1+ KB


In [ ]:
okt = Okt()
for i in range(len(X)):#X를 각각 형태로 분리 하여 저장
  X[i] = okt.morphs(X[i])
print(X) 

0                            [윈썸, 우드, 롤, 업, 테이블]
1                        [모노, 그램, 휘, 게, 체어, 블랙]
2                        [모노, 그램, 휘, 게, 체어, 레드]
3                      [모노, 그램, 휘, 게, 체어, 네이비]
4                                [모노, 그램, 체어, 세트]
                              ...                        
1783    [여자, 오로라, 영롱, 아크릴, 컬러, 세트, 호, 마이카, 반지]...
1784         [피닉스, 시계, 줄, 시계, 밴드, 아크릴, 밴드]
1785           [피닉스, 시계, 줄, 시계, 밴드, 가죽, 밴드]
1786       [피닉스, 시계, 줄, 시계, 밴드, 우레, 탄, 밴드]
1787    [남자, 시계, 메탈, 시계, 오토, 매틱, 시계, 손목시계]...
Name: title, Length: 1788, dtype: object


In [ ]:
stopwords = pd.read_csv('/content/datasets/stopwords2.csv')

In [ ]:
for i in range(len(X)): #각각의 X를 처리하기 위한 for문
    result = []
    for j in range(len(X[i])):#형태소 분리한 X의 단어 개수만큼 for문
        if len(X[i][j]) > 1: #i번째 X에서 j번째 각각의 단어길이가 1보다 크고
            # print(X[i][j])
            if X[i][j] not in list(stopwords['stopword']):
            #i번째 X에서 j번째 단어에 불용어가 포함되어 있지 않으면  
                result.append(X[i][j])
    X[i] = ' '.join(result) #공백기준으로 result하나로 합침
print(X)      

0                             윈썸 우드 테이블
1                               모노 그램 체어
2                          모노 그램 체어 레드
3                        모노 그램 체어 네이비
4                               모노 그램 체어
                         ...                  
1783            오로라 영롱 아크릴 마이카 반지
1784         피닉스 시계 시계 밴드 아크릴 밴드
1785           피닉스 시계 시계 밴드 가죽 밴드
1786                피닉스 시계 시계 밴드 밴드
1787    시계 메탈 시계 오토 매틱 시계 손목시계
Name: title, Length: 1788, dtype: object


In [ ]:
tokened_X = token.texts_to_sequences(X)

print(tokened_X[0])

[36, 8]


In [ ]:
X_pad = pad_sequences(tokened_X, 13)#27은 max값
print(X_pad[:10])

[[    0     0     0     0     0     0     0     0     0     0     0    36
      8]
 [    0     0     0     0     0     0     0     0     0     0   929  2621
     22]
 [    0     0     0     0     0     0     0     0     0   929  2621    22
    136]
 [    0     0     0     0     0     0     0     0     0   929  2621    22
    378]
 [    0     0     0     0     0     0     0     0     0     0   929  2621
     22]
 [    0     0     0     0     0     0     0   929  2621   328  1337    25
    290]
 [    0     0     0     0     0     0     0     0     0     0     0   137
   3882]
 [    0     0     0     0     0     0     0     0     0     0     0  5962
  10134]
 [    0     0     0     0     0     0     0     0     0     0     0     0
   4048]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0]]


In [ ]:
model = load_model('/content/10x10_classfication_0.9141626358032227.h5')

In [ ]:
predict = model.predict(X_pad)

print(predict[0])

[2.1008077e-07 3.1854913e-07 6.2642684e-05 1.6608346e-09 8.4612668e-07
 1.0882920e-06 9.9907792e-01 2.7727583e-06 8.5165899e-04 2.4918118e-06]


In [ ]:
result = np.argmax(predict[0])
print(result) #가장 큰 결과값
print(category[result]) #Politics 가장 많은 개수

6
Furniture


In [ ]:
predict_category = []
for pred in predict:
    #가장 값이 큰 predict의 인덱스 받아오기
    predict_category.append(category[np.argmax(pred)])
print(predict_category)    

['Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'FasionGoods', 'Baby,Kids', 'Deco', 'FasionGoods', 'FasionGoods', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Furniture', 'Food', 'Furniture', 'Furniture', 'Furniture', 'F

In [ ]:
print(Y)

0       Furniture
1       Furniture
2       Furniture
3       Furniture
4       Furniture
          ...    
1783      Jewelry
1784      Jewelry
1785      Jewelry
1786      Jewelry
1787      Jewelry
Name: category, Length: 1788, dtype: object


In [ ]:
test_data_ne['predict'] = predict_category#predict이란 카테고리 추가
print(test_data_ne.head())

                   title   category    predict
0       윈썸 우드 테이블  Furniture  Furniture
1         모노 그램 체어  Furniture  Furniture
2    모노 그램 체어 레드  Furniture  Furniture
3  모노 그램 체어 네이비  Furniture  Furniture
4         모노 그램 체어  Furniture  Furniture


In [ ]:
test_data_ne['OX'] = 0
print(test_data_ne)

                                       title   category    predict  OX
0                           윈썸 우드 테이블  Furniture  Furniture   0
1                             모노 그램 체어  Furniture  Furniture   0
2                        모노 그램 체어 레드  Furniture  Furniture   0
3                      모노 그램 체어 네이비  Furniture  Furniture   0
4                             모노 그램 체어  Furniture  Furniture   0
...                                      ...        ...        ...  ..
1783          오로라 영롱 아크릴 마이카 반지    Jewelry    Jewelry   0
1784       피닉스 시계 시계 밴드 아크릴 밴드    Jewelry    Jewelry   0
1785         피닉스 시계 시계 밴드 가죽 밴드    Jewelry    Jewelry   0
1786              피닉스 시계 시계 밴드 밴드    Jewelry    Jewelry   0
1787  시계 메탈 시계 오토 매틱 시계 손목시계    Jewelry    Jewelry   0

[1788 rows x 4 columns]


In [ ]:
for i in range(len(test_data_ne.predict)):
    if test_data_ne.category[i] == test_data_ne.predict[i]:
        test_data_ne.OX[i] = 'O'
    else:
        test_data_ne.OX[i] = 'X'

print(test_data_ne.head())

                   title   category    predict OX
0       윈썸 우드 테이블  Furniture  Furniture  O
1         모노 그램 체어  Furniture  Furniture  O
2    모노 그램 체어 레드  Furniture  Furniture  O
3  모노 그램 체어 네이비  Furniture  Furniture  O
4         모노 그램 체어  Furniture  Furniture  O


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
print(test_data_ne.OX.value_counts() / len(test_data_ne.OX))

O    0.808725
X    0.191275
Name: OX, dtype: float64


In [ ]:
test_data_ne.iloc[-30:,0:3]

,title,category,predict
1758,체인 스터링 커플 팔찌,Jewelry,Jewelry
1759,체인 스터링 성용 커플 팔찌,Jewelry,Jewelry
1760,스터링 코인 목걸이,Jewelry,Jewelry
1761,스터링 하트 발찌,Jewelry,Jewelry
1762,스터링 스네이크 체인 발찌,Jewelry,Jewelry
1763,스터링 레이어 체인 팔찌,Jewelry,Jewelry
1764,스터링 믹스 체인 발찌,Jewelry,Jewelry
1765,스터링 스틸 체인 발찌,Jewelry,Jewelry
1766,스터링 스틸 체인 팔찌,Jewelry,Jewelry
1767,스터링 체인 목걸이,Jewelry,Jewelry


In [ ]:
test_data_ne.to_csv('test_data_ne.csv', encoding='utf-8-sig')    
print('저장완료')

저장완료
